In [ ]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 9.00MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 20.6MB/s]


In [ ]:
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_df = pd.read_csv("train_data.csv")
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].to_numpy()

In [ ]:
test_df = pd.read_csv("test_data.csv")
X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target'].to_numpy()

# Pretrained Word2Vec

In [ ]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
voc = set(model_w2v.index_to_key)
def document_vector(doc):
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_w2v[doc], axis=0)

In [ ]:
train_w2v = []
for review in X_train['clean_text']:
    train_w2v.append(document_vector(review))

In [ ]:
test_w2v = []
for review in X_test['clean_text']:
    test_w2v.append(document_vector(review))

In [ ]:
np.array(train_w2v).shape

(2842, 300)

In [ ]:
X_train_w2v = X_train.drop(['clean_text'], axis=1).to_numpy()
X_test_w2v = X_test.drop(['clean_text'], axis=1).to_numpy()

In [ ]:
X_train_w2v = np.concatenate((X_train_w2v, train_w2v), axis=1)
X_test_w2v = np.concatenate((X_test_w2v, test_w2v), axis=1)

In [ ]:
print(X_train_w2v.shape)
print(X_test_w2v.shape)

(2842, 409)
(711, 409)


# Training and Testing

In [ ]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [ ]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(alpha=2), scaler.fit_transform(X_train_w2v), scaler.fit_transform(X_test_w2v), y_train, y_test)

{'Accuracy': 0.7313642756680732,
 'Precision': 0.7225433526011561,
 'Recall': 0.7246376811594203,
 'F1-Score': 0.7235890014471781}

In [ ]:
training_and_prediction(GaussianNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7060478199718706,
 'Precision': 0.6634615384615384,
 'Recall': 0.8,
 'F1-Score': 0.7253613666228645}

In [ ]:
training_and_prediction(BernoulliNB(alpha=10), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.6779184247538678,
 'Precision': 0.6421568627450981,
 'Recall': 0.7594202898550725,
 'F1-Score': 0.6958831341301461}

## SVM

In [ ]:
training_and_prediction(SVC(kernel='poly', C=2), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7623066104078763,
 'Precision': 0.7328042328042328,
 'Recall': 0.8028985507246377,
 'F1-Score': 0.7662517289073306}

## Random Forest

In [ ]:
training_and_prediction(RandomForestClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7355836849507735,
 'Precision': 0.7223796033994334,
 'Recall': 0.7391304347826086,
 'F1-Score': 0.7306590257879655}

## Logistic Regression

In [ ]:
training_and_prediction(LogisticRegression(max_iter=5000, solver='saga'), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7651195499296765,
 'Precision': 0.7486033519553073,
 'Recall': 0.7768115942028986,
 'F1-Score': 0.7624466571834992}

## Gradient Boosting

In [ ]:
training_and_prediction(XGBClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7623066104078763,
 'Precision': 0.7458100558659218,
 'Recall': 0.7739130434782608,
 'F1-Score': 0.759601706970128}

## Deep Learning Techniques

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

### ANN

In [ ]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, activation='relu', input_shape=(X_train_w2v.shape[1],)))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_w2v, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_w2v, y_test))
    y_pred_dense = model_dense.predict(X_test_w2v)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [ ]:
for num_neurons_layer1 in [256, 128, 64]:
  for num_neurons_layer2 in [128, 64, 32]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

23/23 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.0}, {'Accuracy': 0.7637130801687764, 'Precision': 0.797979797979798, 'Recall': 0.6869565217391305, 'F1-Score': 0.7383177570093458}]
23/23 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.7327707454289732, 'Precision': 0.7170868347338936, 'Recall': 0.7420289855072464, 'F1-Score': 0.7293447293447293}]
23/23 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7524613220815752, 'Precision': 0.7632398753894081, 'Recall': 0.7101449275362319, 'F1-Score': 0.7357357357357357}]
23/23 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.7665260196905767, 'Precision': 0.7594202898550725, 'Recall': 0.759420289855